In [ ]:
# load the env
import os
from dotenv import load_dotenv
load_dotenv()

import logging
import sys

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output

from llama_index import (
    KnowledgeGraphIndex,
    LLMPredictor,
    ServiceContext,
    SimpleDirectoryReader,
)
from llama_index.storage.storage_context import StorageContext
from llama_index.graph_stores import NebulaGraphStore
from llama_index.llms import OpenAI

from IPython.display import Markdown, display


# define LLM
# NOTE: at the time of demo, text-davinci-002 did not have rate-limit errors
llm = OpenAI(temperature=0, model="text-davinci-002")
service_context = ServiceContext.from_defaults(llm=llm, chunk_size=512)

# Create a NebulaGraph (version 3.5.0 or newer) cluster with:
# Option 0 for machines with Docker: `curl -fsSL nebula-up.siwei.io/install.sh | bash`
# Option 1 for Desktop: NebulaGraph Docker Extension https://hub.docker.com/extensions/weygu/nebulagraph-dd-ext

# If not, create it with the following commands from NebulaGraph's console:
# CREATE SPACE llamaindex(vid_type=FIXED_STRING(256), partition_num=1, replica_factor=1);
# :sleep 10;
# USE llamaindex;
# CREATE TAG entity(name string);
# CREATE EDGE relationship(relationship string);
# :sleep 10;
# CREATE TAG INDEX entity_index ON entity(name(256));

# Creating a NebulaGraph Cluster (Version 3.5.0 or Newer)

## Using Docker (Machines with Docker Installed)
Option 0: Run the following command:
```bash
curl -fsSL nebula-up.siwei.io/install.sh | bash
```

## Desktop Installation
Option 1: Install NebulaGraph Docker Extension from [Docker Hub](https://hub.docker.com/extensions/weygu/nebulagraph-dd-ext).

## Manual Setup via NebulaGraph Console
If the above options are not applicable, manually create the NebulaGraph space, tags, edges, and indexes using the NebulaGraph console:

1. Create the space named `llamaindex`:
```plaintext
CREATE SPACE llamaindex(vid_type=FIXED_STRING(256), partition_num=1, replica_factor=1);
:sleep 10;
USE llamaindex;
```

2. Define a tag named `entity` with a property named `name`:
```plaintext
CREATE TAG entity(name string);
```

3. Define an edge named `relationship`:
```plaintext
CREATE EDGE relationship(relationship string);
:sleep 10;
```

4. Create an index for the `name` property of the `entity` tag:
```plaintext
CREATE TAG INDEX entity_index ON entity(name(256));
```

Adjust the parameters (such as `vid_type`, `partition_num`, `replica_factor`, etc.) based on your specific requirements or environment setup.
```

This Markdown structure provides a step-by-step guide for creating a NebulaGraph cluster using Docker or manual setup via the NebulaGraph console. Adjust the content and formatting as needed for your documentation.

In [ ]:
os.environ["NEBULA_USER"] = "root"
os.environ["NEBULA_PASSWORD"] = "nebula"  # default is "nebula"
os.environ[
    "NEBULA_ADDRESS"
] = "127.0.0.1:9669"  # assumed we have NebulaGraph installed locally

space_name = "llamaindex"
edge_types, rel_prop_names = ["relationship"], [
    "relationship"
]  # default, could be omit if create from an empty kg
tags = ["entity"]  # default, could be omit if create from an empty kg

In [ ]:
graph_store = NebulaGraphStore(
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
)
storage_context = StorageContext.from_defaults(graph_store=graph_store)

In [ ]:
documents = SimpleDirectoryReader("data").load_data()

In [ ]:
kg_index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    max_triplets_per_chunk=10,
    service_context=service_context,
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
    include_embeddings=True,
)

In [ ]:
from llama_index.query_engine import KnowledgeGraphQueryEngine

from llama_index.storage.storage_context import StorageContext
from llama_index.graph_stores import NebulaGraphStore


In [ ]:
query_engine = KnowledgeGraphQueryEngine(
    storage_context=storage_context,
    service_context=service_context,
    llm=llm,
    verbose=True,
)

In [ ]:
response = query_engine.query(
    "you question on the data?",
)
display(Markdown(f"<b>{response}</b>"))